In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
class Matrix_Equation:
    def __init__(self, N):
        self.__N = N
        self.__A = np.eye(N)
        self.__y = np.ones(N)
        self.__x = np.zeros(N)

    def get_N(self):
        return self.__N

    def get_A(self):
        return self.__A

    def set_A(self, A:np.array):
        self.__A = A
        pass    
    
    def get_y(self):
        return self.__y

    def set_y(self, y:np.array):
        self.__y = y
        pass

    def get_x(self):
        return self.__x
    
    def set_x(self, x):
        self.__x = x
        pass

# LU - разложение

Это представление матрицы в виде $A = L U$, где $L$ - нижняя треугольная матрица, $U$ - верхняя треугольная матрица. LU-разложение существует только в том случае, когда матрица $A$ обратима, а все ведущие (угловые) главные миноры матрицы $A$ невырождены.

Алгоритм решения:

1) Вначале надо найти разложение матрицы $A = a_{ij}$ на $L = l_{ij}$ и $U = u_{ij}$:

    $u_{1j} = a_{1j} \text{ , } j = 1,..., n$

    $l_{j1} = \dfrac{a_{j1}}{u_{11}} \text{ , } j = 2, ..., n$

    Если $i = 2, ..., n$:

    $u_{ij} = a_{ij} - \sum_{k = 1}^{i} l_{ik} u_{kj} \text{ , } j = i, ..., n$

    $l_{ji} = \dfrac{1}{u_{ii}} (a_{ji} - \sum_{k = 1}^{i} l_{jk} u_{ki}) \text{ , } j = i+1, ..., n$

2) Затем задача решеается в 2 этапа:

    $\bullet$ $L U x = y \Rightarrow U x = L^{-1} y$

    $\bullet$ $U x = L^{-1} y \Rightarrow x = U^{-1} L^{-1} y$ 

In [3]:
class LU_Solver(Matrix_Equation):
    def __init__(self, N):
        super().__init__(N)
        self.__L = np.zeros((N, N))
        self.__U = np.zeros((N, N))        

    def __str__(self):
        return 'LU_Solver'

    def get_L(self):
        return self.__L

    def get_U(self):
        return self.__U
    
    @staticmethod
    def LU_decomposition(A:np.array):
        n = len(A)
        L = np.zeros((n, n))
        U = np.zeros((n, n))
        
        for i in range(n):
            # Upper Triangular
            for k in range(i, n):
                sum = 0
                for j in range(i):
                    sum += (L[i, j] * U[j, k])
                U[i, k] = A[i, k] - sum
            
            # Lower Triangular
            for k in range(i, n):
                if (i == k):
                    L[i, i] = 1
                else:
                    sum = 0
                    for j in range(i):
                        sum += (L[k, j] * U[j, i])
                    L[k, i] = (A[k, i] - sum) / U[i, i]
        
        return L, U

    @staticmethod
    def L_Solver(L:np.array, y:np.array):
        n = len(y)
        x = np.zeros(n)
        for i in range(n):
            sum = 0
            for j in range(i):
                sum += L[i, j] * x[j]        
            x[i] = 1/L[i, i] * (y[i] - sum)
        return x

    @staticmethod
    def U_Solver(U:np.array, y:np.array):
        n = len(y)
        x = np.zeros(n)
        for i in range(n-1, -1, -1):
            sum = 0
            for j in range(n-1, i, -1):
                sum += U[i, j] * x[j]        
            x[i] = 1/U[i, i] * (y[i] - sum)
        return x  

    def LU_Solver(self):
        self.__L, self.__U = self.LU_decomposition(self.get_A())
        self.set_x(self.U_Solver(self.__U, self.L_Solver(self.__L, self.get_y())))
        return self.get_x()
    
    def time_of_LU_Solver(self):
        start = time.time()
        self.LU_Solver()
        print(f"LU Solver time : {time.time() - start} \n")
        pass

# Метод прогонки

Работает для 3 - диагональной матрицы, для которой выполнено условие диагонального преобладания. 

$$a_{m} x_{m - 1} + b_{m} x_{m} + c_{m} x_{m + 1}= y_{m} \text{ , } m \in 1, ..., n$$

Алгоритм решения:

1) Сначала ищем $P_{1} \text{ и } Q_{1}$ по формулам: $P_{1}=-\dfrac{c_{1}}{b_{1}}, \quad Q_{1}=\dfrac{y_{1}}{b_{1}}$

2) Затем ищем остальные $P_{m}$, $Q_{m}$ по формулам: 

    $P_{m+1}=-\dfrac{c_{m}}{b_{m}+a_{m} P_{m}}, \quad Q_{m+1}=\dfrac{y_{m} - a_{m} Q_{m}}{b_{m}+a_{m} P_{m}}$

3) Затем ищем $x_{n}$ по формуле: $x_{n} = \dfrac{y_{n} - a_{n} Q_{n}}{b_{n}+a_{n} P_{n}}$

4) Решение восстанавливается при помощи прогоночного соотношения: 

    $x_{m} = P_{m+1} x_{m+1} + Q_{m+1}$

In [4]:
class Tridiagonal_Matrix_Solver(Matrix_Equation):
    def __init__(self, N):
        super().__init__(N)
        self.__a = np.zeros(N - 1)
        self.__b = np.ones(N)
        self.__c = np.zeros(N - 1)

    def __str__(self):
        return 'Tridiagonal_Matrix_Solver'

    def get_a(self):
        return self.__a

    def get_b(self):
        return self.__b

    def get_c(self):
        return self.__c

    @staticmethod
    def tridiagonal_matrix_algorithm(a:np.array, b:np.array, c:np.array, y:np.array):
        N = len(y)
        P = np.zeros(N-1)
        Q = np.zeros(N-1)
        x = np.zeros(N)    
        P[0] = - c[0]/b[0]
        Q[0] =  y[0]/b[0]
        for m in range(N-2):
            P[m+1] = - c[m]/(b[m] + a[m]*P[m])
            Q[m+1] = (y[m] - a[m]*Q[m])/(b[m] + a[m]*P[m])
        x[-1] = (y[-1] - a[-1]*Q[-1])/(b[-1] + a[-1]*P[-1])
        for m in range(N-2,-1,-1):
            x[m] = P[m]*x[m+1]+Q[m]
        return x

    def tridiagonal_matrix_decomposition(self):
        for i in range(self.get_N()-1):
            self.__a[i] = self.get_A()[i+1, i]
            self.__b[i] = self.get_A()[i, i]
            self.__c[i] = self.get_A()[i, i+1]
        self.__b[self.get_N()-1] = self.get_A()[self.get_N()-1, self.get_N()-1]
        pass

    def tridiagonal_matrix_Solver(self):
        self.tridiagonal_matrix_decomposition()
        self.set_x(self.tridiagonal_matrix_algorithm(self.__a, self.__b, self.__c, self.get_y()))
        return self.get_x()

    def time_of_tridiagonal_matrix_Solver(self):
        start = time.time()
        self.tridiagonal_matrix_Solver()
        print(f"tridiagonal matrix Solver time : {time.time() - start} \n")
        pass

# Разложение Холецкого

Это представление симметричной положительно определённой матрицы $A$ в виде $A = L L^{T}$, где $L$ - нижняя треугольная матрица со строго положительными элементами на диагонали. Разложение Холецкого всегда существует и единственно для любой симметричной положительно определённой матрицы. По своей сути оно представляет $L U$ разложение симметричной матрицы, поэтому и алгоритм решения совпадает с таковым из 1 пункта.

Выпишем формулы для элементов нижнетреугольной матрицы: 

$\begin{aligned} l_{11} & =\sqrt{a_{11}} \\ l_{j 1} & =\frac{a_{j 1}}{l_{11}}, \quad j \in[2, n] \\ l_{i i} & =\sqrt{a_{i i}-\sum_{p=1}^{i-1} l_{i p}^2}, \quad i \in[2, n] \\ l_{j i} & =\frac{1}{l_{i i}}\left(a_{j i}-\sum_{p=1}^{i-1} l_{i p} l_{j p}\right), \quad i \in[2, n-1], j \in[i+1, n]\end{aligned}$

In [5]:
class Cholesky_Solver(Matrix_Equation):

    def __init__(self, N):
        super().__init__(N)
        self.__L_Cholesky = np.zeros(N)

    def get_L_Cholesky(self):
        
        return self.__L_Cholesky

    @staticmethod
    def Cholesky_decomposition(A:np.array):
        n = len(A)
        L = np.zeros((n, n))
        
        L[0, 0] = np.sqrt(A[0, 0])
        L[1:, 0] = A[1:, 0] / L[0, 0]

        for i in range(1, n):
            L[i, i] = np.sqrt(A[i, i] - np.sum(L[i, :i]*L[i, :i]))
            for j in range(i+1, n):
                    L[j, i] = (A[j, i] - np.sum(L[i, :i]*L[j, :i])) / L[i, i]
        return L

    @staticmethod
    def L_Solver(L:np.array, y:np.array):
        n = len(y)
        x = np.zeros(n)
        for i in range(n):
            sum = 0
            for j in range(i):
                sum += L[i, j] * x[j]        
            x[i] = 1/L[i, i] * (y[i] - sum)
        return x

    @staticmethod
    def U_Solver(U:np.array, y:np.array):
        n = len(y)
        x = np.zeros(n)
        for i in range(n-1, -1, -1):
            sum = 0
            for j in range(n-1, i, -1):
                sum += U[i, j] * x[j]        
            x[i] = 1/U[i, i] * (y[i] - sum)
        return x  

    def Cholesky_Solver(self):
        self.__L_Cholesky = self.Cholesky_decomposition(self.get_A())
        self.set_x(self.U_Solver(np.transpose(self.__L_Cholesky), self.L_Solver(self.__L_Cholesky, self.get_y())))
        return self.get_x()
    
    def time_of_Cholesky_Solver(self):
        start = time.time()
        self.Cholesky_Solver()
        print(f"Cholesky_Solver time : {time.time() - start} \n")
        pass


# QR - разложение

$A = Q R$, где $Q$ - ортогональная матрица, а $R$ - верхнетреугольная матрица.

Алгоритм:

1) Сначала ортогонализируем столбцы матрицы $A$ при помощи алгоритма ортогонализации Грама - Шмидта, полученные векторы и будут составлять матрицу $Q$;

2) Далее находим верхнетреугольную матрицу $R = Q^{-1} A = Q^{T} A$

3) Решение получается в 2 этапа:

    $\bullet$ $Q R x = y \Rightarrow R x = Q^{-1} y = Q^{T} y$

    $\bullet$ $R x = Q^{T} y \Rightarrow x = R^{-1} Q^{T} y$ 

In [6]:
class QR_Solver(Matrix_Equation):

    def __init__(self, N):
        super().__init__(N)
        self.__Q = np.zeros((N, N))
        self.__R = np.zeros((N, N))

    def get_Q(self):
        return self.__Q
    
    def get_R(self):
        return self.__R

    @staticmethod
    def QR_decomposition(A:np.array):
        m, n = A.shape
        Q = np.eye(m)
        R = np.copy(A)

        for j in range(n):

            if (j + 1 == m):
                break
            
            # Вычисляем отражение Хаусхолдера для j-й колонки матрицы R
            normx = np.linalg.norm(R[j:, j])
            if R[j, j] < 0:
                normx = -normx

            u = R[j:, j] - normx * np.eye(len(R[j:, j]))[:, 0]
            v = u/np.linalg.norm(u)

            # Применяем отражение Хаусхолдера к матрицам Q и R
            R[j:, :] -= 2 * np.outer(v, np.dot(v, R[j:, :]))
            Q[:, j:] -= 2 * np.dot(Q[:, j:], np.outer(v, v))
        return Q, R
    
    @staticmethod
    def Q_Solver(Q:np.array, y:np.array):
        return Q.T @ y
    
    @staticmethod
    def R_Solver(R:np.array, y:np.array):
        n = len(y)
        x = np.zeros(n)
        for i in range(n-1, -1, -1):
            sum = 0
            for j in range(n-1, i, -1):
                sum += R[i, j] * x[j]        
            x[i] = 1/R[i, i] * (y[i] - sum)
        return x 
    
    def QR_Solver(self):
        self.__Q, self.__R = self.QR_decomposition(self.get_A())
        self.set_x(self.R_Solver(self.__R, self.Q_Solver(self.__Q, self.get_y())))
        return self.get_x()
    
    def time_of_QR_Solver(self):
        start = time.time()
        self.QR_Solver()
        print(f"QR Solver time : {time.time() - start} \n")
        pass

А теперь соберем все эти солверы по отдельности в один класс:

In [7]:
class Matrix_Solver(LU_Solver, Tridiagonal_Matrix_Solver, Cholesky_Solver, QR_Solver):
 
    def __init__(self, N):
        super().__init__(N)

    def time_of_algorithms(self):
        self.time_of_LU_Solver()
        self.time_of_tridiagonal_matrix_Solver()
        self.time_of_Cholesky_Solver()
        self.time_of_QR_Solver()

In [8]:
N = 100
A = np.eye(N) * 2
for i in range(N - 1):
    A[i+1, i] = 1
    A[i, i+1] = 1

In [9]:
s = Matrix_Solver(N)
s.set_A(A)
x1 = s.LU_Solver()
x2 = s.tridiagonal_matrix_Solver()
x3 = s.Cholesky_Solver()
x4 = s.QR_Solver()
x = np.linalg.solve(s.get_A(), s.get_y())
s.time_of_algorithms()

LU Solver time : 0.10184311866760254 

tridiagonal matrix Solver time : 0.0003750324249267578 

Cholesky_Solver time : 0.03567075729370117 

QR Solver time : 0.007336139678955078 

